- 找出当日涨停的所有股票，不含新股
- 找出N日内有涨停过的股票

In [2]:
import tushare as ts
from tushare.util import dateu as du
import pandas as pd
repo_dir = '/home/pi/practise/python_learning/repo'
#有涨停过的股票
registered_st = 'registered_st'

import os

    
#保存到CSV文件
if not os.path.isfile(repo_dir + '/' + du.last_tddate() + '.csv'):
    #一次性获取当前交易所有股票的行情数据（如果是节假日，即为上一交易日，结果显示速度取决于网速）
    df = ts.get_today_all()
    df.to_csv(repo_dir + '/' + du.last_tddate()+'.csv', encoding='utf-8')
else:
    df = pd.read_csv(repo_dir + '/' + du.last_tddate()+'.csv')
    
#当日涨停股
#df[(df.changepercent > 9) & (df.changepercent < 11)]


- code：代码
- name:名称
- changepercent:涨跌幅
- trade:现价
- open:开盘价
- high:最高价
- low:最低价
- settlement:昨日收盘价
- volume:成交量
- turnoverratio:换手率
- amount:成交量
- per:市盈率
- pb:市净率
- mktcap:总市值
- nmc:流通市值

In [11]:
df_limitupeqty_tmp = df[(df.changepercent > 9) & (df.changepercent < 11)]

#看看个数
#df_limitupeqty_tmp['code'].count()

#新股数据
#- code：股票代码
#- name：股票名称
#- ipo_date:上网发行日期
#- issue_date:上市日期
#- amount:发行数量(万股)
#- markets:上网发行数量(万股)
#- price:发行价格(元)
#- pe:发行市盈率
#- limit:个人申购上限(万股)
#- funds：募集资金(亿元)
#- ballot:网上中签率(%)
newst = ts.new_stocks()
#看看个数
#newst['code'].count()

#上市日期30天以内的算新股（1年前或者几个月前的就不算新股了)
df_newst = newst[(newst.issue_date > du.day_last_week(-30)) | (newst.issue_date.isnull()) ]

#去掉新股，等到涨停的股
df_limitupeqty = df_limitupeqty_tmp[ ~(df_limitupeqty_tmp.code.isin(df_newst.code))]

#重置下index的值，
df_limitupeqty.index=range(df_limitupeqty.code.count())



#保存到CSV文件
if not os.path.isfile(repo_dir + '/' + registered_st + '.csv'):
    #从来没执行过，则直接保存
    df_limitupeqty.to_csv(repo_dir + '/' + registered_st +'.csv', columns=['code'], encoding='utf-8')
else:
    #加载现成的csv
    df_existing = pd.read_csv(repo_dir + '/' + registered_st+'.csv')
    #这行需要吗？
    df_existing.drop_duplicates('code')
    #append 到尾部header那一行就不用了
    #df_limitupeqty.to_csv(repo_dir + '/' + registered_st +'.csv', mode='a', header=None, encoding='utf-8')
    df_result_tmp = pd.concat([df_existing, df_limitupeqty],ignore_index=True)
    df_result = df_result_tmp.drop_duplicates('code')
    df_result.to_csv(repo_dir + '/' + registered_st +'.csv', columns=['code'], encoding='utf-8')
#reminder    
#上边的代码有问题，需要去重复才行,不然后执行几行就多几条 -- 已改
#那个获得新取的接口也不对，去年发行的也叫新股，这。。。。看看有啥参数不对-- 已改
#第一段的csv文件生成也有问题，如果是节假日，应该不会需要取数据，也不需要生成新的CSV，但现在是生成了-- 已改，但改得仍然有问题
    

[Getting data:]###########